In [ ]:
from fastcore.utils import *
from fasthtml.common import *
from fasthtml.jupyter import *
from fastlite import *
import fasthtml.components as fc
import httpx

# Creating an App and Visualizing Components

We're going to build up a web app, spending more time on the pieces than we did in lesson 1. For styling, we're going to change it up and use [https://daisyui.com/](https://daisyui.com/).

To include Daisy in our app, we searched 'Daisy CDN' and copied the code for the required links/scripts. Then we converted them to FastHTML fast tags using [h2f.answer.ai](https://h2f.answer.ai/) (also available in code via the `html2ft` function)

In [ ]:
daisy_hdrs = (
    Link(href='https://cdn.jsdelivr.net/npm/daisyui@5', rel='stylesheet', type='text/css'),
    Script(src='https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4'),
    Link(href='https://cdn.jsdelivr.net/npm/daisyui@5/themes.css', rel='stylesheet', type='text/css')
)

By passing these in as headers (`hdrs`) when creating the app, we ensure this CSS/JS will be available on all pages:

In [ ]:
app = FastHTML(hdrs=daisy_hdrs)
rt = app.route

To see things with these styles applied, we'll make a little conveneince function `preview` that shows them with the app's headers, and later lets us preview routes the same way. In the future, `get_preview` will be available in dialoghelper.

In [ ]:
def get_preview(app):
    return partial(HTMX, app=app, host=None, port=None)
preview = get_preview(app)

Here's how we might make a simple component:

In [ ]:
c = Div(P('hi'), style='color:red')
print(c)

<div style="color:red"><p>hi</p></div>


And to see it rendered:

In [ ]:
preview(c)

We usually return HTML from specific routes. These can render whole pages or (as we'll see later) pieces of HTML to be added to existing pages. Here's a route that serves up our demo component on the `/testing` route:

In [ ]:
@rt
def testing(): return Div(P('hi'), style='color:red')

We need a server running to make this actually accessible. This code starts a server on port 8000, which is accessible via your instance's public URL. Once running, you can visit your-url/testing to see it. We'll stick to using `preview` here though - it let's us preview routes too:

In [ ]:
srv = JupyUvi(app=app)

In [ ]:
preview('/testing')

# Interactivity with HTMX

In [ ]:
@rt
def proc_btn(): return f'hi'

@rt
def testbtn():
    return Div(
        Button('click me', hx_post=proc_btn, id='btn', hx_target='#test', cls='btn'),
        Div(id='test')
    )

In [ ]:
preview(testbtn)

**pause**: There'sa lot of new stuff going on here! We can give elements `id`s. We can have them trigger actions like `post`ing to a different route. We can have that route return html, and specify where it is inserted into the page using CSS selectors like `#test` via `hx_target`. Take a moment to let this sink in.

# Reuseable Components + Tidying

Jeremy doesn't like typing Button(..., cls='btn') every time. We want all buttons to have 'btn' as part of their class. (Styling via class names is how much of the web does it these days - including daisyui). So, we can create our own Button that always adds the appropriate class:

In [ ]:
# Create simple wrapper
def Button(*args, cls='', **kwargs): return fc.Button(*args, cls=f'btn {cls}', **kwargs)


In [ ]:
@rt
def testbtn():
    return Div(
        Button('click me', hx_post=proc_btn, id='btn', hx_target='#test'),
        Div(id='test')
    )

In [ ]:
preview(testbtn)

Same functionality as before, but a little cleaner. 

Now we want an input to type our code into. Looking at DaisyUI, their inputs have an `input` class, with the example given being:

```html
<input type="text" placeholder="Type here" class="input" />
```

In [ ]:
# Create simple wrapper
def Input(*args, cls='', **kwargs): return fc.Input(*args, cls=f'input {cls}', **kwargs)

In [ ]:
preview(Input(type='text', placeholder='Type here'))

I just created both input and button components above. I'd like to create some kind of function where I could basically say, “Hey, create a new function for Button that uses the class btn.” How could we go about doing that? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

You can create a factory function that generates these wrappers:

```python
def mk_comp(comp, dflt_cls):
    def _f(*args, cls='', **kwargs): return comp(*args, cls=f'{dflt_cls} {cls}', **kwargs)
    return _f
```

Then you can use it like this to recreate your components:

```python
Button = mk_comp(fc.Button, 'btn')
Input = mk_comp(fc.Input, 'input')
preview(Input(type='text', placeholder='Type here'))
```

How about having it automatically generate the button symbol instead of just returning it? I'd like to just call `mk_comp('Button', 'btn')` and have it work.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

```python
def mk_comp(name, dflt_cls):
    comp = getattr(fc, name)
    def _f(*args, cls='', **kwargs): return comp(*args, cls=f'{dflt_cls} {cls}', **kwargs)
    globals()[name] = _f
```

Now you can just call:

```python
mk_comp('Button', 'btn')
mk_comp('Input', 'input')
preview(Input(type='text', placeholder='Type here'))
```

In [ ]:
def mk_comp(name, compcls):
    comp = getattr(fc, name)
    def wrapper(*args, cls='', **kwargs): return comp(*args, cls=f'{compcls} {cls}', **kwargs)
    globals()[name] = wrapper

In [ ]:
mk_comp('Button', 'btn')
mk_comp('Input', 'input')

In [ ]:
preview(Button('Test button', cls='btn-primary'))

In [ ]:
preview(Input(type='text', placeholder='Type here'))

In [ ]:
@rt
def getinp(): return Input(type='text', placeholder='Type here')

In [1]:
str(p(getinp))

NameError: name 'p' is not defined

In [ ]:
preview(getinp)

You can see we're starting with just the little pieces we need, and tinkering with ideas like creating the wrapper when we find pieces that seem like they could be improved.

# Getting Data in a Route

To customise what a route returns, we can have it take arguments. As long as you specify a type, FastHTML handles extracting them from the request. There's lots of different ways to pass them, but here's the key bit we'll need for this example: getting the input from the form:

In [ ]:
@rt
def proc_inp(inp:str): return f'hi {inp}'

@rt
def testinp():
    return Form(hx_post=proc_inp, hx_target='#test')(
        Input(type='text', placeholder='Type here', id='inp'),
        Button('say hi', type='submit'),
        Div(id='test')
    )

In [ ]:
preview(testinp())

In [ ]:
@rt
def proc_inp(inp:str): return P(f'hi {inp}')

@rt
def testinp():
    return Form(hx_post=proc_inp, hx_target='#test', hx_swap='afterbegin')(
        Input(type='text', placeholder='Type here', id='inp'),
        Button('say hi', type='submit'),
        Div(id='test')
    )

What if we want to clear the input as well as returning the greeting? One option is to use something in HTMX called an out-of-band swap:

In [ ]:
@rt
def proc_inp(inp:str): 
    return P(f'hi {inp}'), Input(type='text', placeholder='Type here', id='inp')

In [ ]:
@rt
def proc_inp(inp:str): 
    return P(f'hi {inp}'), Input(type='text', placeholder='Type here', id='inp', hx_swap_oob='outerHTML')

In [ ]:
def nameinp(oob=False):
    kw = dict(hx_swap_oob='true') if oob else {}
    return Input(type='text', placeholder='Type here', id='inp', **kw)

@rt
def proc_inp(inp:str): 
    return P(f'hi {inp}'), nameinp(True)

@rt
def testinp():
    return Form(hx_post=proc_inp, hx_target='#test', hx_swap='afterbegin')(
        nameinp(),
        Button('say hi', type='submit'),
        Div(id='test')
    )

Again, we're refactoring things, trying out ideas...

# Code Execution

What if we used the shell we saw in lesson 2 to let you submit code and see the result?

In [ ]:
from toolslm.shell import get_shell

In [ ]:
sh = get_shell()

In [ ]:
sh.run_cell('1+1')

<ExecutionResult object at 75185eac2720, execution_count=None error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 75185eac26f0, raw_cell="1+1" transformed_cell="1+1
" store_history=False silent=False shell_futures=True cell_id=None> result=2>

In [ ]:
sh.run_cell('1+1').result

2

In [ ]:
res = sh.run_cell('print("hi")')

In [ ]:
res.stdout

'hi\n'

In [ ]:
def ex(s):
    res = sh.run_cell(s)
    return res.result if res.result else res.stdout

In [ ]:
ex('print("hi")')

'hi\n'

In [ ]:
ex('1+1')

2

In [ ]:
@rt
def proc_inp(inp:str): 
    res = ex(inp)
    return P(f"{inp}: {res}"), nameinp(True)

In [ ]:
preview(testinp)

# Building Up A UI

Looking at the DaisyUI [chat example](https://daisyui.com/components/chat/?lang=en), I (Johno now) thought it would be fun to make a pretty little chat UI around this idea. Plan: 
- Have it in a page, with a title
- Central Chat Area
- Input at the bottom
- Chat bubbles for the input + response appear once you submit.

Starting with the HTML copied from the daisy UI chat example (collapsed because all we want is the minimum piece to get this going):

In [ ]:
from fasthtml.common import html2ft

print(html2ft("""<div class="chat chat-start">
  <div class="chat-bubble chat-bubble-primary">What kind of nonsense is this</div>
</div>
<div class="chat chat-start">
  <div class="chat-bubble chat-bubble-secondary">
    Put me on the Council and not make me a Master!??
  </div>
</div>
<div class="chat chat-start">
  <div class="chat-bubble chat-bubble-accent">
    That's never been done in the history of the Jedi.
  </div>
</div>
<div class="chat chat-start">
  <div class="chat-bubble chat-bubble-neutral">It's insulting!</div>
</div>
<div class="chat chat-end">
  <div class="chat-bubble chat-bubble-info">Calm down, Anakin.</div>
</div>
<div class="chat chat-end">
  <div class="chat-bubble chat-bubble-success">You have been given a great honor.</div>
</div>
<div class="chat chat-end">
  <div class="chat-bubble chat-bubble-warning">To be on the Council at your age.</div>
</div>
<div class="chat chat-end">
  <div class="chat-bubble chat-bubble-error">It's never happened before.</div>
</div>""")) # The daisyUI color chat bubble example

Div(
    Div('What kind of nonsense is this', cls='chat-bubble chat-bubble-primary'),
    cls='chat chat-start'
)

Div(
    Div('Put me on the Council and not make me a Master!??', cls='chat-bubble chat-bubble-secondary'),
    cls='chat chat-start'
)

Div(
    Div("That's never been done in the history of the Jedi.", cls='chat-bubble chat-bubble-accent'),
    cls='chat chat-start'
)

Div(
    Div("It's insulting!", cls='chat-bubble chat-bubble-neutral'),
    cls='chat chat-start'
)

Div(
    Div('Calm down, Anakin.', cls='chat-bubble chat-bubble-info'),
    cls='chat chat-end'
)

Div(
    Div('You have been given a great honor.', cls='chat-bubble chat-bubble-success'),
    cls='chat chat-end'
)

Div(
    Div('To be on the Council at your age.', cls='chat-bubble chat-bubble-warning'),
    cls='chat chat-end'
)

Div(
    Div("It's never happened before.", cls='chat-bubble chat-bubble-error'),
    cls='chat chat-end'
)


We can see how they make messages using the `chat-bubble` class, with `chat-start` or `chat-end` determining which side the messages show, and colors based on extra classes like `chat-bubble-primary`.

In [ ]:
preview(Div(
    Div(
        Div('Hello World', cls='chat-bubble chat-bubble-primary'),
        cls='chat chat-start'
    ),
    Div(
        Div('Hello.', cls='chat-bubble chat-bubble-info'),
        cls='chat chat-end'
    ), style="max-width:400px"))

I added the max width since I didn't like seeing them super far apart by default on my nice high-res screen.

At this point I get impatient, and like to make a minimal version of the page I'm picturing in my head to then modify. You might first want to refactor the chat pieces - but at this point I'm not even sure I want to use them. So:

Build a little demo page with a central chat area, with a form at the bottom that submits code, and set it up so that the code appears as a chat-end message and the response shows up as a chat-start one underneath it.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

```python
@rt
def proc_code(code:str):
    res = ex(code)
    return (Div(Div(code, cls='chat-bubble chat-bubble-info'), cls='chat chat-end'),
            Div(Div(str(res), cls='chat-bubble chat-bubble-primary'), cls='chat chat-start'),
            Input(type='text', placeholder='Enter code...', id='code', hx_swap_oob='true'))

@rt
def chatdemo():
    return Div(
        H1('Code Chat', cls='text-3xl font-bold mb-4'),
        Div(id='chat', cls='mb-4', style='max-width:600px'),
        Form(hx_post=proc_code, hx_target='#chat', hx_swap='beforeend')(
            Input(type='text', placeholder='Enter code...', id='code', cls='input-bordered w-full'),
            Button('Run', type='submit', cls='btn-primary mt-2')
        ),
        cls='p-4'
    )

preview(chatdemo)
```

In [ ]:
@rt
def proc_code(code:str):
    res = ex(code)
    return (Div(Div(code, cls='chat-bubble chat-bubble-info'), cls='chat chat-end'),
            Div(Div(str(res), cls='chat-bubble chat-bubble-primary'), cls='chat chat-start') if res else "",
            Input(type='text', placeholder='Enter code...', id='code', hx_swap_oob='true'))

@rt
def chatdemo():
    return Div(
        H1('Code Chat', cls='text-3xl font-bold mb-4'),
        Div(id='chat', cls='mb-4', style='max-width:600px'),
        Form(hx_post=proc_code, hx_target='#chat', hx_swap='beforeend')(
            Input(type='text', placeholder='Enter code...', id='code'),
            Button('Run', type='submit', cls='btn-primary')
        ),
        cls='p-4'
    )

preview(chatdemo)

Not bad! Take some time to read the code. We show the heading, a chat area, and the input form. Submitting sends the code to `proc_code` which sends back a bubble for the code, a bubble for the response, and a cleared input via an oob swap.

There are pieces I right away don't like, so we can start changing them here:

Explain what changes I'd need to make to:
- Make the title centered
- Make the code input full width (apart from the button)
- Add a nice border with rounded corners to the chat area, and have it be fixed 400px height with scrolling if more messages come in.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

To make those changes:

- **Centered title**: Add `text-center` to the H1 cls
- **Full width input**: Put the Input and Button in a flex container with `flex gap-2`, and add `flex-1` to the Input cls
- **Chat area styling**: Add `border rounded-lg p-4 h-[400px] overflow-y-auto` to the chat Div cls

Here's the updated version:

```python
@rt
def chatdemo():
    return Div(
        H1('Code Chat', cls='text-3xl font-bold mb-4 text-center'),
        Div(id='chat', cls='mb-4 border rounded-lg p-4 h-[400px] overflow-y-auto', style='max-width:600px'),
        Form(hx_post=proc_code, hx_target='#chat', hx_swap='beforeend', cls='flex gap-2')(
            Input(type='text', placeholder='Enter code...', id='code', cls='flex-1'),
            Button('Run', type='submit', cls='btn-primary')
        ),
        cls='p-4'
    )

preview(chatdemo)
```

In [ ]:
@rt
def proc_code(code:str):
    res = ex(code)
    return (Div(Div(code, cls='chat-bubble chat-bubble-info'), cls='chat chat-end'),
            Div(Div(str(res), cls='chat-bubble chat-bubble-primary'), cls='chat chat-start') if res else "",
            Input(type='text', placeholder='Enter code...', id='code', hx_swap_oob='true', cls='flex-1'))

@rt
def chatdemo():
    return Div(
        H1('Code Chat', cls='text-3xl font-bold mb-4 text-center'),
        Div(id='chat', cls='mb-4 border rounded-lg p-4 h-[400px] overflow-y-auto'),
        Form(hx_post=proc_code, hx_target='#chat', hx_swap='beforeend', cls='flex gap-2')(
            Input(type='text', placeholder='Enter code...', id='code', cls='flex-1'),
            Button('Run', type='submit', cls='btn-primary')
        ),
        cls='p-4', style='max-width:600px'
    )

preview(chatdemo)

Explain flex-1 and flex gap-2

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

`flex-1` makes an element grow to fill available space in a flex container. It's shorthand for `flex: 1 1 0%`, which means the element can grow and shrink, and starts with a base size of 0. So your input will take up all remaining space after the button.

`gap-2` adds spacing between flex items (or grid items). The `2` is a Tailwind spacing unit equal to 0.5rem (8px). So `gap-2` puts 8px of space between your input and button without needing margins.

OK, that's much better! But this is about the limit in terms of size of thing that's nice to iterate on in this way - back to Jeremy to do this more incrementally...